## 라이브러리 임포트

In [35]:
import numpy as np
import pandas as pd
import tensorflow as tf

# 파트 1. 데이터 전처리

## 데이터셋 임포트

데이터에 있는 내용은 아래와 같다.

1. 줄 번호
2. 고객 아이디
3. 성(姓)
4. 신용점수
5. 지역
6. 성별
7. 나이
8. 은행 근무년도
9. 은행에 돈이 얼마나 있는지
10. 은행과 관련된 제품이 얼마나 있는지 (예: 카드, 대출)
11. 카드가 있는지
12. 내부 데이터를 이용한 활동여부 판단
13. 연봉
14. 나갔는지에 대한 여부

이 데이터를 이용해서 우리가 만들 목표는 다음과 같다.  
"과연 어떤 고객이 은행을 나갈 것인가?"  
14번째 컬럼이 타겟이 된다.

세번째 까지의 컬럼과 마지막 컬럼은 타겟을 예측하는데   
도움이 되지 않기 때문에 제외한다.

In [36]:
dataset = pd.read_csv('/content/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [37]:
dataset.head(1)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,101348.88,1


In [38]:
dataset.shape

(10000, 14)

In [39]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [40]:
print(y)

[1 0 1 ... 1 1 0]


## 범주형 데이터 인코딩

성별은 라벨 인코딩을 한다.  
국가는 원핫 인코딩을 한다.

라벨 인코딩은 그 컬럼 안에서 숫자가 나눠진다.  
원핫 인코딩은 새로운 컬럼을 만들어서 0과 1로 표현한다.

One-Hot Encoding은 순서가 없고
고유값 개수가 많지 않을 때 사용한다.

Label Encoding은 (사장, 대리)같은 직급처럼
순서가 의미가 있을 때 사용한다.



In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,2])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

0,1,2 까지는 국가이다. 3,4 는 성별이다.

국가 순서는 프랑스, 독일, 스페인순이고  
성별은 여성, 남성순이다.

순서는 영어 스펠링 순서이다.

In [42]:
print(X[0])

[1.0 0.0 0.0 1.0 0.0 619 42 2 0.0 1 1 1 101348.88]


## 트레이닝 셋과 테스트 셋 나누기

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

##데이터 정규화

StandardScaler는 평균과 표준편차를 이용하여  
데이터를 정규화 한다.

In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [45]:
X_test

array([[-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 ,  1.61085707],
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  0.49587037],
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         0.9687384 , -0.42478674],
       ...,
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
        -1.03227043,  0.71888467],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 , -1.54507805],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
        -1.03227043,  1.61255917]])

# 파트 2. 인공 신경망 만들기

## 인공신경망 초기화

In [46]:
ann = tf.keras.models.Sequential()

##인풋 레이어와 첫 번째 히든 레이어 추가

In [47]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

##두 번째 히든 레이어 추가

In [48]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## 아웃풋 레이어 추가

In [49]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

sigmoid, sigmoid, sigmoid 보다  
relu, relu, sigmoid가 결과가 더 잘나온다.

그리고 units = 6 인이유는 다른 걸 넣어도  
비슷하게 나오기 때문에 그렇다.  

아웃풋은 하나니까 unit = 1이다.  
자세하게 알고 싶으면 딥러닝을 더 공부해야한다.

# 파트 3. 인공신경망 트레이닝

## 인공신경망 컴파일링

인간이 쓴 코드를 컴퓨터가 알 수 있도록 바꿔주는 작업

아담으로 최적화를 한다.

loss는 Cost function과 유사한 역할을 한다.  
결과가 2개일 때는 binary_crossentropy를 사용하고  
3개 이상의 결과에서는 categorical_crossentropy를 사용한다.

metrics는 실제 화면상에 출력되는 output을 표현한다.  
여기서는 정확성을 알고 싶기에 accuracy를 넣었다.



In [50]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

##인공신경망 트레이닝

batch_size의 default는 32이다.  
epochs는 반복을 뜻한다.

In [51]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5356 - accuracy: 0.7865
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4620 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4443 - accuracy: 0.7966
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4350 - accuracy: 0.8005
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4287 - accuracy: 0.8027
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4236 - accuracy: 0.8077
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4187 - accuracy: 0.8099
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4126 - accuracy: 0.8138
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4045 - accuracy: 0.8202
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3958 - accura

# 파트 4. 예측과 모델평가

## 관찰값 예측



지역: 독일  
성별: 여  
신용점수: 500  
나이: 25   
은행 근무년도: 2  
은행에 돈이 얼마나 있는지: 30000  
은행과 관련된 제품이 얼마나 있는지 (예: 카드, 대출): 2      
카드가 있는지: No  
내부 데이터를 이용한 활동여부 판단: No  
연봉:  30000

In [56]:
print(ann.predict(sc.transform([[0, 1, 0, 1, 0, 500, 25, 2, 30000, 2, 0, 0, 30000]])) > 0.5)

[[False]]


여기서 0.5가 뜻하는 것은 임계점이다.  
즉, 0.5가 넘어가면 exit를 한다는 뜻이고  
그것이 아니면 남는다는 뜻이다.

## 테스트 셋 예측 확인

In [61]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## 분류 모델 성능 평가 지표

In [62]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1489  106]
 [ 184  221]]


0.855